In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


running k-means on cuda:0..


[running kmeans]: 8it [00:00, 42.95it/s, center_shift=0.000075, iteration=8, tol=0.000100]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

# TODO: auto sim cutoff needed
decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=2)
decomp.load()

Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.3.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.4.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.5.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from 

In [3]:
decomp.lattice_scores[0].shape

(50, 50)

In [4]:
from typing import List
import utils
importlib.reload(utils)
modified_lattice = decomp.lattice_scores.copy()

# TODO: have function which does something like fixing from the a specific neuron?
# Ie have the sub-lattice with only **related** neurons

def cutoff_lattice(lattice: List[List[List[float]]], related_cutoff = 1):
    print(lattice[0].sum())
    r = [(layer > related_cutoff) * layer for layer in lattice]
    print(r[0].sum())
    return r

def create_sublattice(lattice, layer: int, idx_in_layer: int, related_cutoff = 10):
	with_cutoff = cutoff_lattice(lattice, related_cutoff)

LAST_FEAT = 20
modified_lattice = cutoff_lattice(decomp.lattice_scores, related_cutoff=15)

# utils.top_k_dag_paths_dynamic(modified_lattice, k=10_000)

10436.000003378393
200.502825928319


In [5]:
# TODO:
""""
We need to combine the 
"""

import numpy as np
from utils import get_random_cutoff

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
# score_path = [39, 14, 2, 3, 4]
score_path = [1, 1, 2, 3, 4, 5]
"""
Paths which seem to have meaning

score_path = [1, 1, 2, 3, 4]
Very strong for Q: . Specifically on the space
"""

to_score = [get_random_cutoff(d, 100) for d in ds][:300]
scores = decomp.score(
    to_score,
    score_path=score_path,
    weighting_per_layer=np.array([1, 1, 1, 1, 1, 1])
)


GOT EMBEDS (6, 5262, 512)
(5262, 6, 512) [1, 1, 2, 3, 4, 5]


In [6]:
from IPython.core.display import display, HTML
import math
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!
act_simp = scores_reord
act_simp = [[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 100))]] for tok in s] for s in scores_reord]
# TODO sep fun
from circuitsvis.utils.render import RenderedHTML, render
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))

/tmp/ipykernel_33380/3908721336.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Isolate Specific Neurons

In [7]:
import cluster_model
importlib.reload(cluster_model)

LAYER = 3
NODE = 10
# TODO: this ain't right!! We need specific paths!!
ret = cluster_model.restrict_to_related_path(decomp.lattice_scores, LAYER, NODE, n_paths=100)
len(ret), ret

(100,
 [([35, 16, 10, 10, 31, 8], 418.76821920884277),
  ([35, 16, 10, 10, 8, 22], 415.53488057437016),
  ([35, 16, 10, 10, 8, 44], 415.511392029322),
  ([35, 16, 10, 10, 8, 31], 415.5025872796391),
  ([35, 16, 10, 10, 8, 11], 415.45330660149875),
  ([35, 16, 10, 10, 8, 47], 415.42202576663067),
  ([35, 16, 10, 10, 8, 6], 415.4111877099781),
  ([35, 16, 10, 10, 8, 4], 415.40546014875827),
  ([35, 16, 10, 10, 18, 44], 415.4043359242346),
  ([35, 16, 10, 10, 18, 22], 415.3918553249673),
  ([35, 16, 10, 10, 8, 10], 415.3712790590066),
  ([35, 16, 10, 10, 18, 31], 415.3667145095965),
  ([35, 16, 10, 10, 8, 13], 415.3649209039281),
  ([35, 16, 10, 10, 8, 7], 415.34845456080757),
  ([35, 16, 10, 10, 13, 22], 415.34123379280163),
  ([35, 16, 10, 10, 13, 44], 415.3343826906489),
  ([35, 16, 10, 10, 18, 11], 415.31837547568966),
  ([35, 16, 10, 10, 22, 22], 415.3183341613035),
  ([35, 16, 10, 10, 8, 26], 415.3152726552794),
  ([35, 16, 10, 10, 14, 22], 415.3148741963955),
  ([35, 16, 10, 10, 18

## Try different scoring method

In [18]:
importlib.reload(cluster_model)

score_path = [0, 16, 10, 10, 31, 8]
# TODO: embeds to here...
importlib.reload(cluster_model)
_, top_ds, scores = cluster_model.score_for_neuron(decomp, to_score[:200], 0, score_path)
top_ds

EMBEDS LEN 6
GOT EMBEDS (6, 278, 512)
(278, 6, 512) [0, 16, 10, 10, 31, 8]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Try more path like method

In [11]:
score_path = [35, 16, 10, 10, 31, 8]

"""
Paths which seem to have meaning

score_path = [1, 1, 2, 3, 4]
Very strong for Q: . Specifically on the space

score_path_path = [3, 15, 19, 11, 12]
Very strong for Q: . when context is about *programming*
"""

to_score = [d[:100] for d in ds][:600]
scores = decomp.score(
    to_score,
    score_path=score_path,
    weighting_per_layer=np.array([10, 1, 1, 1, 1, 1]) # TODO: how do we weight???
)

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!
act_simp = scores_reord
act_simp = [[[[0 if math.isnan(tok * 10) else int(tok * 100)]] for tok in s] for s in scores_reord]
# TODO sep fun
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))

GOT EMBEDS (6, 5500, 512)
(5500, 6, 512) [35, 16, 10, 10, 31, 8]
